In [1]:
import requests
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import copy
import time
import random
import threading

from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
from binance.helpers import round_step_size
from futures_sign_test import send_signed_request, send_public_request
from cred_test import KEY, SECRET

In [2]:
symbol='BTCUSDT'
client = Client(KEY, SECRET, testnet=True)

maxposition=0.01 #1 = 100%
quantity=str(maxposition/2)
profit_percent=0.02
stop_percent=profit_percent/1.7
pointer=str(random.randint(1000, 9999))

In [3]:
# Hedge mode 
client.futures_get_position_mode()

{'dualSidePosition': True}

In [4]:
# Find all opened positions

def get_opened_positions(symbol):
    status = client.futures_account()
    positions=pd.DataFrame(status['positions'])
    a = positions[positions['symbol']==symbol]['notional'].astype(float).tolist()[1]
    b = positions[positions['symbol']==symbol]['notional'].astype(float).tolist()[2]
    open_positions=positions[positions['symbol']==symbol]
    long_position=float(open_positions[open_positions['positionSide']=='LONG']['entryPrice'])
    short_position=float(open_positions[open_positions['positionSide']=='SHORT']['entryPrice'])
    profit = float(status['totalUnrealizedProfit'])
    balance = round(float(status['totalWalletBalance']),2)
    if a>0:
        pos_long = "long"
    else:
        pos_long = ""
    if b<0:
        pos_short = "short"
    else: 
        pos_short = ""
    return([pos_long,pos_short,profit,balance,long_position,short_position])

In [5]:
get_opened_positions(symbol)

['', '', 0.0, 2682.06, 0.0, 0.0]

In [6]:
# Close all orders 

def check_and_close_orders(symbol):
    global isStop 
    a=client.futures_get_open_orders(symbol=symbol)
    if len(a)>0:
        isStop = False
        client.futures_cancel_all_open_orders(symbol=symbol)

In [7]:
check_and_close_orders(symbol)

In [8]:
def get_symbol_price(symbol):
    prices = client.get_all_tickers()
    df=pd.DataFrame(prices)
    return float(df[ df['symbol']==symbol]['price'])

In [9]:
#получить tick size
data = client.futures_exchange_info()
tick_size = 0.0
the_symbol = "BTCBUSD"

found = False
info = data['symbols']
for s in range(len(info)):
    if info[s]['symbol'] == the_symbol:
        filters = info[s]['filters']
        for f in range(len(filters)):
            if filters[f]['filterType'] == 'PRICE_FILTER':
                tick_size = float(filters[f]['tickSize'])
                found = True
                break
        break
if found:
    print(tick_size)        
else:
    print(f"tick_size not found for {the_symbol}")

0.01


In [10]:
#Stop loss LONG 
def open_sl_long(symbol,quantity):
    try:

        position=get_opened_positions(symbol)
        entry_price=position[4]
        stop_price_trigger=str(round(float(entry_price)*(1-stop_percent), 2))
        stop_price=str(round(float(stop_price_trigger)*1.001, 1))
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "SELL",
                    "positionSide": "LONG",
                    "type": "STOP",
                    "quantity": quantity,
                    "timeInForce":"GTE_GTC",
                    "tickSize": "0.1",
                    "stopPrice": stop_price_trigger,
                    "price": stop_price
                }
            ]
        }

        open_position_sl_long_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
        long_sl_opened=float(open_position_sl_long_response[0]['price'])
        print('LONG sl order opened price:'+str(long_sl_opened))
        return(long_sl_opened)
    
    except :
        print('\n\nsl long not open...')
        

In [11]:
#Take Profit LONG 

def open_tp_long(symbol,quantity):
    try:

        position=get_opened_positions(symbol)
        entry_price=position[4]
        take_price_trigger=str(round(float(entry_price)*(1+profit_percent), 2))
        take_price=str(round(float(take_price_trigger)*1.001, 1))
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "SELL",
                    "positionSide": "LONG",
                    "type": "TAKE_PROFIT",
                    "quantity": quantity,
                    "timeInForce":"GTE_GTC",
                    "tickSize": "0.1",
                    "stopPrice": take_price_trigger,
                    "price": take_price
                }
            ]
        }
        open_position_tp_long_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
        long_tp_opened=float(open_position_tp_long_response[0]['price'])
        print('LONG tp order opened price:'+str(long_tp_opened))
        return(long_tp_opened)
        
    except :
        print('\n\nTP long not open...')

In [12]:
#Открыть позицию LONG 

def open_position_long(symbol,quantity):
    sprice=get_symbol_price(symbol)
    close_price=str(round(sprice*(1+0.005),1))
    params = {
        "batchOrders": [
            {
                "symbol":symbol,
                "side": "BUY",
                "positionSide": "LONG",
                "type": "LIMIT",
                "quantity": quantity,
                "timeInForce":"GTC",
                "tickSize": "0.10",
                "price": close_price        
            }
        ]
    }
    open_position_long_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
    long_opened=float(open_position_long_response[0]['price'])
    print('LONG order opened price:'+str(long_opened))
    return(long_opened)

In [13]:
#Открыть sl&tp long если есть позиция

def long_tp_sl_opening(symbol):
    orders=client.futures_get_open_orders(symbol=symbol)
    orders_df=pd.DataFrame(orders)
    if orders_df.empty:
        x = threading.Thread(target=open_tp_long, args=(symbol,quantity))
        x.start()
        y = threading.Thread(target=open_sl_long, args=(symbol,quantity))
        y.start()
    else:
        order_long=orders_df[orders_df['positionSide']=='LONG']
        long_tp=order_long[order_long['origType']=='TAKE_PROFIT']['price']
        long_sl=order_long[order_long['origType']=='STOP']['price']
        if (long_tp+long_sl).empty:
            x = threading.Thread(target=open_tp_long, args=(symbol,quantity))
            x.start()
            y = threading.Thread(target=open_sl_long, args=(symbol,quantity))
            y.start()
        else:
            print('LONG SL TP iteration continious..')

In [14]:
#Stop loss SHORT 

def open_sl_short(symbol,quantity):
    try:
        position=get_opened_positions(symbol)
        entry_price=position[5]
        stop_price_trigger=str(round(float(entry_price)*(1+stop_percent), 2))
        stop_price=str(round(float(stop_price_trigger)*1.001, 1))
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "BUY",
                    "positionSide": "SHORT",
                    "type": "STOP",
                    "quantity": quantity,
                    "timeInForce":"GTE_GTC",
                    "tickSize": "0.001",
                    "stopPrice": stop_price_trigger,
                    "price": stop_price
                }
            ]
        }
        
        open_position_sl_short_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
        short_sl_opened=float(open_position_sl_short_response[0]['price'])
        print('SHORT sl order opened price:'+str(short_sl_opened))
        return(short_sl_opened)
    
    except :
        print('\n\nSL short not open...')


In [15]:
#Take Profit SHORT

def open_tp_short(symbol,quantity):
    try:
        position=get_opened_positions(symbol)
        entry_price=position[5]
        take_price_trigger=str(round(float(entry_price)*(1-profit_percent), 2))
        take_price=str(round(float(take_price_trigger)*1.001, 1))
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "BUY",
                    "positionSide": "SHORT",
                    "type": "TAKE_PROFIT",
                    "quantity": quantity,
                    "timeInForce":"GTE_GTC",
                    "tickSize": "0.01",
                    "stopPrice": take_price_trigger,
                    "price": take_price
                }
            ]
        }

        open_position_tp_short_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
        short_tp_opened=float(open_position_tp_short_response[0]['price'])
        print('SHORT tp order opened price:'+str(short_tp_opened))
        return(short_tp_opened)
    
    except :
        print('\n\nTP short not open...')


In [16]:
#Открыть позицию SHORT

def open_position_short(symbol,quantity):
    sprice=get_symbol_price(symbol)
    close_price=str(round(sprice*(1-0.001),1))
    params = {
        "batchOrders": [
            {
                "symbol":symbol,
                "side": "SELL",
                "positionSide": "SHORT",
                "type": "LIMIT",
                "quantity": quantity,
                "timeInForce":"GTC",
                "tickSize": "0.10",
                "price": close_price        
            }
        ]
    }
    open_position_short_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
    short_opened=float(open_position_short_response[0]['price'])
    print('SHORT order opened price:'+str(short_opened))
    return(short_opened)

In [17]:
#Открыть sl&tp SHORT если есть позиция
       
def short_tp_sl_opening(symbol):
    orders=client.futures_get_open_orders(symbol=symbol)
    orders_df=pd.DataFrame(orders)
    if orders_df.empty:
        x = threading.Thread(target=open_tp_short, args=(symbol,quantity))
        x.start()
        y = threading.Thread(target=open_sl_short, args=(symbol,quantity))
        y.start()
    else:
        order_short=orders_df[orders_df['positionSide']=='SHORT']
        short_tp=order_short[order_short['origType']=='TAKE_PROFIT']['price']
        short_sl=order_short[order_short['origType']=='STOP']['price']
        if (short_tp+short_sl).empty:
            x = threading.Thread(target=open_tp_short, args=(symbol,quantity))
            x.start()
            y = threading.Thread(target=open_sl_short, args=(symbol,quantity))
            y.start()
        else:
            print('SHORT SL TP iteration continious..')

In [18]:
def check_positions_new(symbol):
    positions=get_opened_positions(symbol)
    if positions[0]+positions[1]=='':
        print('Start new positions')
        x = threading.Thread(target=open_position_short, args=(symbol,quantity))
        x.start()
        y = threading.Thread(target=open_position_long, args=(symbol,quantity))
        y.start()

In [19]:
def check_positions_ready(symbol):
    positions=get_opened_positions(symbol)
    if positions[0]+positions[1]=='longshort':
        print('Positions ready, sl&tp stage..')
        x = threading.Thread(target=short_tp_sl_opening(symbol))
        x.start()
        y = threading.Thread(target=long_tp_sl_opening(symbol))
        y.start()

In [20]:
def main(step):

    try:
        x = threading.Thread(target=check_positions_new(symbol))
        x.start()
        y = threading.Thread(target=check_positions_ready(symbol))
        y.start()
    except :
        print('\n\nSomething went wrong. Continuing...')

In [28]:
# main(3)

Positions ready, sl&tp stage..
SHORT SL TP iteration continious..
LONG SL TP iteration continious..


In [22]:
telegram_delay=12
bot_token='5454990445:AAGzfeY80oeI-1su0MEIRVuvTzBYIVlZIXU'
chat_id='92428939'

def getTPSLfrom_telegram():
    strr='https://api.telegram.org/bot'+bot_token+'/getUpdates'
    response = requests.get(strr)
    rs=response.json()
    exit=0

    if(len(rs['result'])>0):
        rs2=rs['result'][-1]
        rs3=rs2['message']
        textt=rs3['text']
        datet=rs3['date']        
        if(time.time()-datet)<telegram_delay:
            if 'exit' in textt:
                exit=1
                telegram_bot_sendtext('Right now! Sir')

            # if 'close_pos' in textt:
            #     position=get_opened_positions(symbol)
            #     open_sl=position[0]
            #     quantity=position[1]
            #     print(open_sl,quantity)
            #     close_position(symbol,open_sl,abs(quantity)) #abs - по модулю
    return(exit)

In [23]:
getTPSLfrom_telegram()

0

In [24]:
def telegram_bot_sendtext(bot_message):
    bot_token2 = bot_token
    bot_chatID = chat_id
    send_text = 'https://api.telegram.org/bot' + bot_token2 + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message
    response = requests.get(send_text)
    return response.json()

In [25]:
def prt(message):
    # telegram message
    telegram_bot_sendtext(pointer+': '+message)
    print(pointer+': '+message)

In [26]:
def start_stop():
    starttime=time.time()
    counterr=1
    timeout = time.time() + 60*60*12  # 60 seconds times 60 meaning the script will run for 12 hr
        
    while time.time() <= timeout:
        try:

            print("script continue running at "+time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
            main(counterr)
            counterr=counterr+1
            stop=getTPSLfrom_telegram()
            if stop==1:
                break
            if counterr>3:
                break
                #counterr=1
            time.sleep(10 - ((time.time() - starttime) % 10.0)) # 1 minute interval between each new execution
        except KeyboardInterrupt:
            print('\n\KeyboardInterrupt. Stopping.')
        

In [27]:
start_stop()

script continue running at 2022-10-05 15:27:56
Start new positions
SHORT order opened price:19812.8
LONG order opened price:19931.8
script continue running at 2022-10-05 15:28:06
Positions ready, sl&tp stage..
SHORT sl order opened price:20068.9
SHORT tp order opened price:19438.8
LONG tp order opened price:20241.3
LONG sl order opened price:19611.0
script continue running at 2022-10-05 15:28:16
Positions ready, sl&tp stage..
SHORT SL TP iteration continious..
LONG SL TP iteration continious..
